In [125]:
import json
import logging
import os
import random
from collections.abc import Callable
from copy import deepcopy
from datetime import datetime
from json import dumps
from pathlib import Path
from time import time
from typing import Any, Final, Literal, Unpack

import numpy as np
import structlog
import torch
import torch.nn as nn
from bayes_opt import BayesianOptimization
from numpy import argmax, concatenate, ndarray, vstack, where
from pandas import DataFrame, Series, concat, read_csv, set_option, to_datetime
from scipy.spatial.distance import pdist, squareform
from scipy.stats import loguniform, randint, uniform, wilcoxon
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn.metrics import (
	auc,
	average_precision_score,
	classification_report,
	f1_score,
	precision_recall_curve,
	roc_auc_score,
)
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import RobustScaler
from sklearn.svm import OneClassSVM

set_option("display.max_columns", None)
logger: Final[structlog.stdlib.BoundLogger] = structlog.get_logger(__name__)
type ParamGrid = dict[str, tuple[float | str, ...]]

COLUMNS: Final[list[str]] = [
	"timestamp",
	"activity",
	"heart_rate",
	*[
		f"IMU_{body_part}_{suffix}"
		for body_part in ["hand", "chest", "ankle"]
		for suffix in [
			"temp_C",
			*[
				f"{scalar}_{axis}"
				for scalar in ["acc16g_ms^-2", "acc6g_ms^-2", "gyro_rad/s", "mag_μT"]
				for axis in ["x", "y", "z"]
			],
			*[f"orient_{x}" for x in range(1, 5)],
		]
	],
]
IMU_COLUMNS: Final[list[str]] = [
	col
	for col in COLUMNS
	if col.startswith("IMU_") and "acc6g_ms^-2" not in col and "orient" not in col
]
NUM_TRIALS: Final[int] = 10

# Global variables for scoring
testing_data: DataFrame | None = None
test_targets: Series | None = None

structlog.configure(
	processors=[
		structlog.stdlib.filter_by_level,
		structlog.stdlib.add_logger_name,
		structlog.stdlib.add_log_level,
		structlog.stdlib.PositionalArgumentsFormatter(),
		structlog.processors.TimeStamper(fmt="iso"),
		structlog.processors.StackInfoRenderer(),
		structlog.processors.format_exc_info,
		structlog.processors.UnicodeDecoder(),
		structlog.processors.JSONRenderer(),
	],
	context_class=dict,
	logger_factory=structlog.stdlib.LoggerFactory(),
	wrapper_class=structlog.stdlib.BoundLogger,
	cache_logger_on_first_use=True,
)

In [126]:
def configure_file_logger(filepath: str) -> structlog.BoundLogger:
	"""Configure a file-specific logger using structlog."""

	# Create a specific handler for this file
	file_handler = logging.FileHandler(filepath, mode="a")
	file_handler.setLevel(logging.INFO)

	# Configure structlog with file output
	structlog.configure(
		processors=[
			structlog.stdlib.filter_by_level,
			structlog.stdlib.add_logger_name,
			structlog.stdlib.add_log_level,
			structlog.stdlib.PositionalArgumentsFormatter(),
			structlog.processors.TimeStamper(fmt="iso"),
			structlog.processors.StackInfoRenderer(),
			structlog.processors.format_exc_info,
			structlog.processors.UnicodeDecoder(),
			structlog.processors.JSONRenderer(),
		],
		context_class=dict,
		logger_factory=structlog.stdlib.LoggerFactory(),
		wrapper_class=structlog.stdlib.BoundLogger,
		cache_logger_on_first_use=False,
	)
	# Get the underlying stdlib logger and add the handler
	stdlib_logger = logging.getLogger("hyperparameter_search")
	stdlib_logger.handlers.clear()  # Clear existing handlers
	stdlib_logger.addHandler(file_handler)
	stdlib_logger.setLevel(logging.INFO)

	return structlog.get_logger("hyperparameter_search")

## **NOVELTY DETECTION ENSEMBLE DEFINITION**:

In [127]:
class LSTMAutoencoder(nn.Module):
	"""LSTM-based Autoencoder for time-series novelty detection."""

	def __init__(
		self,
		n_features: int,
		sequence_length: int,
		hidden_size: int = 64,
		n_layers: int = 2,
		dropout: float = 0.2,
	):
		super(LSTMAutoencoder, self).__init__()
		self.n_features = n_features
		self.sequence_length = sequence_length
		self.hidden_size = hidden_size
		self.encoder = nn.LSTM(
			input_size=n_features,
			hidden_size=hidden_size,
			num_layers=n_layers,
			batch_first=True,
			dropout=dropout if n_layers > 1 else 0,
		)
		self.decoder = nn.LSTM(
			input_size=hidden_size,
			hidden_size=hidden_size,
			num_layers=n_layers,
			batch_first=True,
			dropout=dropout if n_layers > 1 else 0,
		)
		self.output_layer = nn.Linear(hidden_size, n_features)

	def forward(self, x):
		# Encode
		_, (hidden, cell) = self.encoder(x)
		# Repeat hidden state for decoder
		decoder_input = hidden[-1].unsqueeze(1).repeat(1, self.sequence_length, 1)
		# Decode
		decoder_output, _ = self.decoder(decoder_input, (hidden, cell))
		# Output reconstruction
		return self.output_layer(decoder_output)


class LSTMAutoencoderWrapper:
	"""Wrapper for LSTM Autoencoder to match sklearn API."""

	def __init__(
		self,
		n_features: int,
		sequence_length: int = 100,
		hidden_size: int = 64,
		n_layers: int = 2,
		dropout: float = 0.2,
		epochs: int = 50,
		batch_size: int = 64,
		learning_rate: float = 1e-3,
		device: str | None = None,
	):
		self.n_features = n_features
		self.sequence_length = sequence_length
		self.hidden_size = hidden_size
		self.n_layers = n_layers
		self.dropout = dropout
		self.epochs = epochs
		self.batch_size = batch_size
		self.learning_rate = learning_rate
		self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

		self.model = None
		self.threshold = None

	def _create_sequences(self, data: ndarray) -> ndarray:
		"""Create sliding windows from flat data."""
		data = data.astype(np.float32)
		return np.array(
			[
				data[i : i + self.sequence_length]
				for i in range(len(data) - self.sequence_length + 1)
			],
			dtype=np.float32,  # Explicitly set dtype
		)

	def fit(self, X: DataFrame, y=None):
		"""Train the LSTM Autoencoder on normal data."""
		# Convert to numpy (already scaled by normalize_features)
		sequences = self._create_sequences(X[IMU_COLUMNS].values.astype(np.float32))
		if len(sequences) == 0:
			logger.warning(
				"Not enough data to create sequences. Skipping LSTM-AE training."
			)
			return self

		self.model = LSTMAutoencoder(
			n_features=self.n_features,
			sequence_length=self.sequence_length,
			hidden_size=self.hidden_size,
			n_layers=self.n_layers,
			dropout=self.dropout,
		).to(self.device)

		# Training setup
		criterion = nn.MSELoss()
		optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
		dataloader = torch.utils.data.DataLoader(
			torch.utils.data.TensorDataset(torch.FloatTensor(sequences)),
			batch_size=self.batch_size,
			shuffle=True,
		)
		self.model.train()
		for epoch in range(self.epochs):  # Training loop
			epoch_loss = 0
			for batch in dataloader:
				batch_data = batch[0].to(self.device)
				# Forward pass
				reconstruction = self.model(batch_data)
				loss = criterion(reconstruction, batch_data)
				# Backward pass
				optimizer.zero_grad()
				loss.backward()
				optimizer.step()

				epoch_loss += loss.item()

			if (epoch + 1) % 10 == 0:
				logger.info(
					f"LSTM-AE Epoch {epoch + 1}/{self.epochs} "
					+ f"Loss: {epoch_loss / len(dataloader):.4f}",
				)

		# Treshold on training data (95th percentile of reconstruction errors)
		self.threshold = np.percentile(self._calc_reconstruction_errors(sequences), 95)

		return self

	def _calc_reconstruction_errors(self, sequences: ndarray) -> ndarray:
		"""Compute reconstruction error for sequences."""
		assert self.model is not None, "Model is not trained yet."
		self.model.eval()
		errors = []

		with torch.no_grad():
			for i in range(0, len(sequences), self.batch_size):
				batch_tensor = torch.FloatTensor(sequences[i : i + self.batch_size]).to(
					self.device
				)
				errors.extend(  # MSE per sequence
					torch.mean(
						(batch_tensor - self.model(batch_tensor)) ** 2, dim=(1, 2)
					)
					.cpu()
					.numpy()
				)
		return np.array(errors)

	def predict(self, X: DataFrame) -> ndarray:
		"""Predict: 1 for normal, -1 for anomaly."""
		return where(self.decision_function(X) > self.threshold, -1, 1)

	def decision_function(self, X: DataFrame) -> ndarray:
		"""Return anomaly scores (higher = more anomalous)."""
		sequences = self._create_sequences(X[IMU_COLUMNS].values.astype(np.float32))
		# Data is already scaled - no transformation needed
		if len(sequences) == 0:
			# If not enough data for a sequence, return high anomaly score
			return np.array([self.threshold * 2 if self.threshold else 1.0] * len(X))

		errors = self._calc_reconstruction_errors(sequences)
		# Pad to match original length (for sequences lost at the end)
		if (padding := len(X) - len(errors)) > 0:
			errors = np.concatenate([errors, errors[-1:].repeat(padding)])

		return errors

	def score(self, X: DataFrame, y=None) -> float:
		"""
		Score method for sklearn compatibility.
		Returns negative mean reconstruction error (higher is better).
		"""
		if self.threshold is None:
			return 0.0

		sequences = self._create_sequences(X[IMU_COLUMNS].values.astype(np.float32))
		if len(sequences) == 0:
			return 0.0
		# Return negative error (sklearn maximizes score)
		return -float(np.mean(self._calc_reconstruction_errors(sequences)))

	def get_params(self, deep=True):
		"""
		Get parameters for sklearn compatibility.

		Args:
			deep (bool): Ignored, present for sklearn compatibility. Default to True.
		"""
		return {
			"n_features": self.n_features,
			"sequence_length": self.sequence_length,
			"hidden_size": self.hidden_size,
			"n_layers": self.n_layers,
			"dropout": self.dropout,
			"epochs": self.epochs,
			"batch_size": self.batch_size,
			"learning_rate": self.learning_rate,
		}

	def set_params(self, **params):
		"""Set parameters for sklearn compatibility."""
		for key, value in params.items():
			setattr(self, key, value)
		return self

In [135]:
class NoveltyDetectionEnsemble(BaseEstimator, ClassifierMixin):
	"""Ensemble combining LSTM-AE, Isolation Forest, and One-Class SVM."""

	def __init__(
		self,
		n_features: int,
		# LSTM-AE params
		sequence_length: int = 100,
		hidden_size: int = 64,
		n_layers: int = 2,
		dropout: float = 0.2,
		epochs: int = 50,
		batch_size: int = 64,
		learning_rate: float = 1e-3,
		# Isolation Forest params
		n_estimators: int = 200,
		max_samples: int = 256,
		contamination: float = 0.05,
		# OC-SVM params
		nu: float = 0.05,
		gamma: float | Literal["scale", "auto"] = "scale",
		# Ensemble params
		weights: tuple[float, float, float] = (0.4, 0.3, 0.3),
		**kwargs: dict[str, Any],  # unnused, just for scikit learn compatibility
	):
		self.n_features = n_features
		self.weights = np.array(weights)

		self.lstm_ae = LSTMAutoencoderWrapper(
			n_features=n_features,
			sequence_length=sequence_length,
			hidden_size=hidden_size,
			n_layers=n_layers,
			dropout=dropout,
			epochs=epochs,
			batch_size=batch_size,
			learning_rate=learning_rate,
		)
		self.isolation_forest = IsolationForest(
			n_estimators=n_estimators,
			max_samples=max_samples,
			contamination=contamination,
			random_state=42,
		)
		self.ocsvm = OneClassSVM(nu=nu, gamma=gamma, kernel="rbf")

		self.models = [self.isolation_forest, self.ocsvm]
		self.model_names = ["IsolationForest", "OC-SVM"]

	def fit(self, X: DataFrame, y=None):
		"""Train all models in the ensemble."""
		logger.info("Training ensemble models...")
		for name, model in zip(self.model_names, self.models, strict=True):
			logger.info(f"Training {name}...")
			model.fit(X, y)
		logger.info("Ensemble training complete.")
		return self

	def predict(self, X: DataFrame) -> ndarray:
		"""Predict: 1 for normal, -1 for anomaly."""
		scores = self.decision_function(X)
		return where(scores > np.median(scores), -1, 1)

	def decision_function(self, X: DataFrame) -> ndarray:
		"""Return combined anomaly scores (higher = more anomalous)."""
		return np.average(
			[
				self._normalize_scores(self.lstm_ae.decision_function(X)),
				self._normalize_scores(
					-self.isolation_forest.decision_function(X.values)
				),
				self._normalize_scores(-self.ocsvm.decision_function(X.values)),
			],
			axis=0,
			weights=self.weights,
		)

	def score(self, X: DataFrame, y=None, sample_weight=None) -> float:
		"""
		Score method for sklearn compatibility.
		For novelty detection, return negative mean anomaly score.
		"""
		return -float(np.mean(self.decision_function(X)))

	def _normalize_scores(self, scores: ndarray) -> ndarray:
		"""Normalize scores to [0, 1] range using min-max scaling."""
		if (max_score := np.max(scores)) - (min_score := np.min(scores)) == 0:
			return np.zeros_like(scores)
		return (scores - min_score) / (max_score - min_score)

	def get_params(self, deep=True):
		"""Get all ensemble parameters in flat structure."""
		return {
			"n_features": self.n_features,
			"weights": tuple(self.weights),
			**self.lstm_ae.get_params(deep=False),
			**{
				k: v
				for k, v in self.isolation_forest.get_params(deep=False).items()
				if k in ["n_estimators", "max_samples", "contamination"]
			},
			**{
				k: v
				for k, v in self.ocsvm.get_params(deep=False).items()
				if k in ["nu", "gamma"]
			},
		}

	def set_params(self, **params):
		"""Set parameters and reinitialize models."""
		# Separate parameters by model
		lstm_params = {}
		if_params = {}
		ocsvm_params = {}
		ensemble_params = {}

		for key, value in params.items():
			if key in [
				"sequence_length",
				"hidden_size",
				"n_layers",
				"dropout",
				"epochs",
				"batch_size",
				"learning_rate",
			]:
				lstm_params[key] = value
			elif key in ["n_estimators", "max_samples", "contamination"]:
				if_params[key] = value
			elif key in ["nu", "gamma"]:
				ocsvm_params[key] = value
			elif key in ["weights", "n_features"]:
				ensemble_params[key] = value
			else:
				setattr(self, key, value)

		# Update ensemble-level params
		for key, value in ensemble_params.items():
			setattr(self, key, value)
			if key == "weights":
				self.weights = np.array(value)

		# Reinitialize models with new parameters
		if lstm_params or "n_features" in ensemble_params:
			lstm_params["n_features"] = getattr(self, "n_features", self.n_features)
			for key in [
				"sequence_length",
				"hidden_size",
				"n_layers",
				"dropout",
				"epochs",
				"batch_size",
				"learning_rate",
			]:
				if key not in lstm_params:
					lstm_params[key] = getattr(self.lstm_ae, key)

			self.lstm_ae = LSTMAutoencoderWrapper(**lstm_params)

		if if_params:
			current_if_params = self.isolation_forest.get_params()
			current_if_params.update(if_params)
			self.isolation_forest = IsolationForest(
				**current_if_params, random_state=42
			)

		if ocsvm_params:
			current_ocsvm_params = self.ocsvm.get_params()
			current_ocsvm_params.update(ocsvm_params)
			self.ocsvm = OneClassSVM(**current_ocsvm_params, kernel="rbf")

		# Update models list
		self.models = [self.lstm_ae, self.isolation_forest, self.ocsvm]

		return self

## **HYPER-PARAMS OPTIMIZATION METHODS IMPLEMENTAION**:

In [136]:
def score_function(
	model: NoveltyDetectionEnsemble,
	Train: DataFrame,
	test: Series,
	testing_data: DataFrame,
) -> dict:
	"""
	Objective function to maximize, calcs the F1 score on the test set.
	Now works with ensemble model.

	Args:
		model (NoveltyDetectionEnsemble): Ensemble model to eval
		Train (DataFrame): train data, only for API compliance
		test (Series): true targets (True for novel, False for normal)
		testing_data (DataFrame): test feature data

	Returns:
		dict: Evaluation metrics
	"""
	# Get predictions and anomaly scores
	f1 = f1_score(test, where(model.predict(testing_data) == -1, True, False))
	# Get decision scores (higher values = more anomalous)
	anomaly_scores = model.decision_function(testing_data)
	precision, recall, _ = precision_recall_curve(test, anomaly_scores)
	metrics = {
		"f1_score": float(f1),
		"avg_precision": float(average_precision_score(test, anomaly_scores)),
		"auc_pr": float(auc(recall, precision)),
		"auc_roc": float(roc_auc_score(test, anomaly_scores)),
		"params": model.get_params(),
		"datetime": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
	}
	logger.info(f"Evaluation metrics: {metrics}")
	return metrics

In [137]:
class SimulatedAnnealingSearch:
	"""
	Custom Simulated Annealing implementation for hyperparameter optimization.

	Simulated Annealing Search:
	- Temperature-based acceptance: Accepts worse solutions with decreasing probability
	- Adaptive parameter perturbation: Different strategies for continuous vs discrete parameters
	- Cooling schedule: Exponential cooling with configurable rate
	- Neighbor generation: Smart parameter space exploration
	"""

	def __init__(
		self,
		param_space: dict[str, list[int | float | str]],
		n_iter: int = 100,
		initial_temp: float = 1.0,
		cooling_rate: float = 0.95,
		min_temp: float = 0.01,
		random_state: int = 42,
	):
		self.param_space = param_space
		self.n_iter = n_iter
		self.initial_temp = initial_temp
		self.cooling_rate = cooling_rate
		self.min_temp = min_temp
		self.random_state = random_state
		self.best_params_ = None
		self.best_score_ = -np.inf
		self.cv_results_ = {"mean_test_score": []}

	def _sample_params(self) -> dict:
		"""Sample random parameters from the parameter space."""
		try:
			return {
				key: (
					values.rvs(random_state=self.random_state)
					if hasattr(values, "rvs")  # scipy distribution
					else random.choice(values)
					if isinstance(values, (list, tuple))  # Ensure it's a sequence
					else values  # Use the value directly if it's not sampleable
				)
				for key, values in self.param_space.items()
			}
		except Exception as exc:
			print(self.param_space)
			logger.error("Error sampling parameters.")
			raise exc

	def _neighbor_params(self, current_params: dict) -> dict:
		"""Generate neighboring parameters by slightly modifying current ones."""
		neighbor = deepcopy(current_params)

		# Choose a random parameter to modify
		param_to_modify = random.choice(list(self.param_space.keys()))

		if hasattr(self.param_space[param_to_modify], "rvs"):  # continuous parameter
			if param_to_modify == "nu":
				# For nu, stay within bounds [0.001, 1.0]
				current_val = neighbor[param_to_modify]
				neighbor[param_to_modify] = np.clip(
					current_val + np.random.normal(0, 0.05 * current_val), 0.001, 1.0
				)
			elif param_to_modify == "gamma":
				# For gamma, use log-space perturbation
				neighbor[param_to_modify] = 10 ** np.clip(
					np.log10(neighbor[param_to_modify]) + np.random.normal(0, 0.1),
					-4,
					1,
				)  # 1e-4 to 10
			elif param_to_modify == "tol":
				# For tolerance, use log-space perturbation
				neighbor[param_to_modify] = 10 ** np.clip(
					np.log10(neighbor[param_to_modify]) + np.random.normal(0, 0.1),
					-6,
					-1,
				)  # 1e-6 to 1e-1
		else:  # discrete parameter
			neighbor[param_to_modify] = random.choice(self.param_space[param_to_modify])

		return neighbor

	def _evaluate_params(self, params: ParamGrid, X: DataFrame, y: Series) -> float:
		"""Evaluate parameter configuration using cross-validation."""
		print(params)
		return np.mean(
			cross_val_score(
				NoveltyDetectionEnsemble(**params), X, y, cv=4, scoring=score_function
			)
		).astype(float)

	def fit(self, X: DataFrame, y: Series):
		"""Fit the simulated annealing search."""
		random.seed(self.random_state)
		np.random.seed(self.random_state)
		# Initialize with random parameters
		current_params = self._sample_params()
		current_score = self._evaluate_params(current_params, X, y)

		self.best_params_ = deepcopy(current_params)
		self.best_score_ = current_score

		temperature = self.initial_temp

		for iteration in range(self.n_iter):
			# Generate neighbor, store the score for cv_results and Accept or reject neighbor
			neighbor_params = self._neighbor_params(current_params)
			neighbor_score = self._evaluate_params(neighbor_params, X, y)
			self.cv_results_["mean_test_score"].append(neighbor_score)

			if neighbor_score > current_score:  # Better solution - always accept
				current_params = neighbor_params
				current_score = neighbor_score
			else:  # Worse solution - accept with probability
				if (
					random.random()
					< np.exp(neighbor_score - current_score / temperature)
					if temperature > 0
					else 0
				):
					current_params = neighbor_params
					current_score = neighbor_score

			if current_score > self.best_score_:  # Update best solution
				self.best_params_ = deepcopy(current_params)
				self.best_score_ = current_score

			# Cool down
			temperature = max(temperature * self.cooling_rate, self.min_temp)

		return self


class GeneticSearch:
	"""
	Custom Genetic Algorithm implementation for hyperparameter optimization.

	Genetic Algorithm Search:
	- Population-based optimization: Maintains diverse parameter sets
	- Tournament selection: Robust parent selection mechanism
	- Uniform crossover: Parameter exchange between parents
	- Adaptive mutation: Random parameter changes with configurable rate
	- Elite preservation: Keeps best solutions across generations
	"""

	def __init__(
		self,
		param_space: dict,
		population_size: int = 20,
		n_generations: int = 10,
		mutation_rate: float = 0.1,
		crossover_rate: float = 0.8,
		elite_size: int = 2,
		random_state: int = 42,
	):
		self.param_space = param_space
		self.population_size = population_size
		self.n_generations = n_generations
		self.mutation_rate = mutation_rate
		self.crossover_rate = crossover_rate
		self.elite_size = elite_size
		self.random_state = random_state
		self.best_params_ = None
		self.best_score_ = -np.inf
		self.cv_results_ = {"mean_test_score": []}

	def _create_individual(self) -> dict:
		"""Create a random individual (parameter set)."""
		return {
			key: (
				values.rvs(random_state=self.random_state)
				if hasattr(values, "rvs")  # scipy distribution
				else random.choice(values)
				if isinstance(values, (list, tuple))  # Ensure it's a sequence
				else values  # Use the value directly if it's not sampleable
			)
			for key, values in self.param_space.items()
		}

	def _crossover(self, parent1: dict, parent2: dict) -> tuple[dict, dict]:
		"""Create two offspring from two parents using uniform crossover."""
		child1, child2 = deepcopy(parent1), deepcopy(parent2)

		for key in parent1.keys():
			if random.random() < 0.5:  # Swap parameter values
				child1[key], child2[key] = child2[key], child1[key]

		return child1, child2

	def _mutate(self, individual: dict) -> dict:
		"""Mutate an individual by randomly changing some parameters."""
		mutated = deepcopy(individual)

		for key in individual.keys():
			if random.random() < self.mutation_rate:
				mutated[key] = (
					self.param_space[key].rvs(random_state=self.random_state)
					if hasattr(self.param_space[key], "rvs")  # continuous parameter
					else random.choice(self.param_space[key])  # discrete parameter
				)
		return mutated

	def _tournament_selection(
		self, population: list, fitness_scores: list, tournament_size: int = 3
	) -> dict:
		"""Select an individual using tournament selection."""
		tournament_indices = random.sample(
			range(len(population)), min(tournament_size, len(population))
		)
		return population[
			tournament_indices[
				np.argmax([fitness_scores[i] for i in tournament_indices])
			]
		]

	def _evaluate_params(self, params: dict, X: DataFrame, y: Series) -> float:
		"""Evaluate parameter configuration using cross-validation."""
		return np.mean(
			cross_val_score(
				NoveltyDetectionEnsemble(**params, kernel="rbf"),
				X,
				y,
				cv=4,
				scoring=score_function,
			)
		).astype(float)

	def fit(self, X: DataFrame, y: Series):
		"""Fit the genetic algorithm search."""
		random.seed(self.random_state)
		np.random.seed(self.random_state)
		population = [self._create_individual() for _ in range(self.population_size)]

		for generation in range(self.n_generations):  # Evaluate fitness
			print(f"Evaluating Generation {generation}")
			fitness_scores = []
			for individual in population:
				score = self._evaluate_params(individual, X, y)
				fitness_scores.append(score)
				self.cv_results_["mean_test_score"].append(score)

				if score > self.best_score_:
					self.best_params_ = deepcopy(individual)
					self.best_score_ = score

			# Create next generation | Elite selection - keep best individuals
			new_population = [
				deepcopy(population[idx])
				for idx in np.argsort(fitness_scores)[-self.elite_size :]
			]
			# Generate offspring
			while len(new_population) < self.population_size:
				# Selection
				parent1 = self._tournament_selection(population, fitness_scores)
				parent2 = self._tournament_selection(population, fitness_scores)
				# Crossover
				if random.random() < self.crossover_rate:
					child1, child2 = self._crossover(parent1, parent2)
				else:
					child1, child2 = deepcopy(parent1), deepcopy(parent2)
				# Mutation
				new_population.extend([self._mutate(child1), self._mutate(child2)])
			# Trim to exact population size
			population = new_population[: self.population_size]

		return self


class BayesianSearch:
	"""Wrapper for Bayesian Optimization to match sklearn API."""

	def __init__(
		self,
		param_bounds: dict,
		n_iter: int = 100,
		random_state: int = 42,
		X_train: DataFrame | None = None,
		y_train: Series | None = None,
	):
		self.param_bounds = param_bounds
		self.n_iter = n_iter
		self.random_state = random_state
		self.X_train = X_train
		self.y_train = y_train
		self.best_params_ = None
		self.best_score_ = -np.inf
		self.cv_results_ = {"mean_test_score": []}

	def _params_to_model_config(self, **params) -> dict:
		"""Convert Bayesian params to model configuration."""
		discrete_params = {  # need rounding
			"sequence_length": int(params["sequence_length"]),
			"hidden_size": int(params["hidden_size"]),
			"n_layers": int(round(params["n_layers"])),
			"epochs": int(params["epochs"]),
			"batch_size": int(params["batch_size"]),
			"n_estimators": int(params["n_estimators"]),
			"max_samples": int(params["max_samples"]),
		}
		continuous_params = {
			"dropout": params["dropout"],
			"learning_rate": params["learning_rate"],
			"contamination": params["contamination"],
			"nu": params["nu"],
			"gamma": 10 ** params["gamma_log"],
		}
		# Weights derived from weight_lstm
		weight_others = (1 - (weight_lstm := params["weight_lstm"])) / 2
		return {
			**discrete_params,
			**continuous_params,
			**{"weights": (weight_lstm, weight_others, weight_others)},
			"n_features": len(IMU_COLUMNS),
		}

	def _objective(self, **params) -> float:
		"""Objective function for Bayesian optimization."""
		# Simple train/test split evaluation (you can modify for CV)
		assert self.X_train is not None, "Training data not set."
		assert self.y_train is not None, "Training Labels not set."
		score = score_function(
			NoveltyDetectionEnsemble(**self._params_to_model_config(**params)).fit(
				self.X_train
			),
			self.X_train,
			self.y_train,
			self.X_train,
		)["f1_score"]
		self.cv_results_["mean_test_score"].append(score)
		return score

	def fit(self, X: DataFrame, y: Series):
		"""Fit using Bayesian optimization."""
		self.X_train = X
		self.y_train = y

		optimizer = BayesianOptimization(
			f=self._objective,
			pbounds=self.param_bounds,
			random_state=self.random_state,
			allow_duplicate_points=True,
		)
		optimizer.maximize(init_points=10, n_iter=self.n_iter)

		assert optimizer.max, "Optimization did not find any maximum. Somehow..."

		self.best_params_ = self._params_to_model_config(**optimizer.max["params"])
		self.best_score_ = optimizer.max["target"]

		return self

## **INCREMENTAL DATASET IMPLEMENTATION**:

In [138]:
class IncrementalDataset:
	"""Manages data splits for incremental/continual learning experiments."""

	def __init__(
		self,
		path: Path,
		initial_activities: int = 3,
		test_size: int = 2,
		activity_order: list[int] | None = None,
		ordering_method: Literal["pca", "statistical", "variance", "frequency"] = "pca",
	):
		self.data, self.labels = self.load_data(path)
		self.df = self.data.merge(self.labels, how="left", on="id")

		self.initial_activities = initial_activities
		self.test_size = test_size

		self.activity_order = (
			self.get_activity_order(self.data, self.labels, method=ordering_method)
			if activity_order is None
			else activity_order
		)
		self.n_activities = len(self.activity_order)
		self.n_folds = (self.n_activities - initial_activities) // test_size

		print("\n📦 Incremental Dataset Configuration:")
		print(f"  Total activities: {self.n_activities}")
		print(f"  Initial training activities: {initial_activities}")
		print(f"  Test size per fold: {test_size}")
		print(f"  Number of folds: {self.n_folds}")
		print(f"  Activity order: {self.activity_order}")

	def read_w_log(self, path: Path, filename: str) -> tuple[DataFrame, str]:
		"""Read and preprocess PAMAP2 data file."""
		print(f"Reading: {filename}", end="\r")
		df = read_csv(os.path.join(path, filename), sep=r"\s+", header=None)
		df.columns = COLUMNS
		return (
			df.loc[:, ~df.columns.str.contains(r"orient|acc6g", regex=True)],
			filename.split(".")[0][-2:],
		)

	def handle_nans(self, df: DataFrame) -> DataFrame:
		"""Handles NaN values in the sensor data with a time-series-aware strategy."""
		df = df.copy()
		for col in IMU_COLUMNS:
			df.loc[:, col] = (
				df[col]
				.ffill(limit=2)
				.interpolate("linear", limit=5, limit_direction="both")
			)
		return df.dropna(subset=IMU_COLUMNS)

	def normalize_features(
		self,
		X_train: DataFrame,
		X_val: DataFrame | None = None,
		X_test: DataFrame | None = None,
		scaler: RobustScaler | None = None,
		force_refit: bool = False,
	) -> tuple[DataFrame, DataFrame | None, DataFrame | None, RobustScaler]:
		"""Normalizes IMU features using RobustScaler."""
		if scaler is None or force_refit:
			scaler = RobustScaler().fit(X_train[IMU_COLUMNS])

		X_train = X_train.copy()
		X_train.loc[:, IMU_COLUMNS] = scaler.transform(X_train[IMU_COLUMNS])

		X_val_norm = None
		if X_val is not None:
			X_val_norm = X_val.copy()
			X_val_norm.loc[:, IMU_COLUMNS] = scaler.transform(X_val[IMU_COLUMNS])

		X_test_norm = None
		if X_test is not None:
			X_test_norm = X_test.copy()
			X_test_norm.loc[:, IMU_COLUMNS] = scaler.transform(X_test[IMU_COLUMNS])

		return X_train, X_val_norm, X_test_norm, scaler

	def load_data(self, base_path: Path) -> tuple[DataFrame, DataFrame]:
		"""Load and preprocess all PAMAP2 protocol data."""
		logger.info("Loading PAMAP2 data...")
		if (data_csv := base_path / "data.csv").exists() and (
			labels_csv := base_path / "labels.csv"
		).exists():
			logger.info("Found preprocessed data files. Loading...")
			return read_csv(data_csv), read_csv(labels_csv)

		data, labels = [], []
		for df, subject in [
			self.read_w_log(base_path, filename)
			for filename in os.listdir(base_path)
			if filename.endswith(".dat")
		]:
			df = self.handle_nans(df[~df["activity"].isin([0, 24])])
			df["subject"] = str(subject)
			df["timestamp"] = to_datetime(df["timestamp"], unit="s").dt.time
			df["id"] = df["subject"] + "_" + df["timestamp"].astype(str)

			data.append(df.drop(columns=["timestamp", "activity", "heart_rate"]))
			labels.append(df[["id", "timestamp", "activity"]])

		data, labels = concat(data), concat(labels)
		data["subject"] = data["subject"].astype("category")
		labels["activity"] = labels["activity"].astype("category")

		data.to_csv("../data/PAMAP2/data.csv", index=False)
		labels.to_csv("../data/PAMAP2/labels.csv", index=False)

		return data, labels

	def get_activity_order(
		self,
		data: DataFrame,
		labels: DataFrame,
		method: Literal["pca", "statistical", "variance", "frequency"] = "pca",
	) -> list[int]:
		"""Order activities by distinctiveness for incremental learning."""
		print(f"\n🔍 Ordering activities for incremental learning using '{method}'...")
		df = data.merge(labels, how="left", on="id")

		if method == "frequency":
			activity_counts = df["activity"].value_counts()
			print("\n📊 Activity ordering (most → least frequent):")
			for i, act in enumerate(ordered := activity_counts.index.tolist(), 1):
				print(f"  {i}. Activity {act}: {activity_counts[act]:,} samples")
			return ordered

		activity_stats = {}
		activity_counts = {}
		for activity in df["activity"].unique():
			activity_data = df[df["activity"] == activity][IMU_COLUMNS]
			activity_counts[activity] = len(activity_data)

			if method == "pca":
				activity_stats[activity] = (
					PCA(n_components=min(10, len(IMU_COLUMNS), len(activity_data)))
					.fit_transform(activity_data)
					.mean(axis=0)
				)
			elif method == "statistical":
				activity_stats[activity] = concatenate(
					[
						activity_data.mean().values,
						activity_data.std().values,
						activity_data.quantile(0.25).values,
						activity_data.quantile(0.75).values,
					]
				)
			elif method == "variance":
				activity_stats[activity] = concatenate(
					[
						activity_data.var().values,
						activity_data.abs().mean().values,
					]
				)

		activities = list(activity_stats.keys())
		distances = squareform(
			pdist(
				vstack([activity_stats[act] for act in activities]), metric="euclidean"
			)
		)
		ordered = []
		remaining = set(range(len(activities)))
		avg_distances = distances.mean(axis=1)
		first_idx = argmax(
			[
				-avg_distances[i] if i in remaining else float("-inf")
				for i in range(len(activities))
			]
		).astype(int)
		ordered.append(first_idx)
		remaining.remove(first_idx)

		while remaining:
			min_dists = [
				min([distances[idx, sel_idx] for sel_idx in ordered])
				if idx in remaining
				else float("inf")
				for idx in range(len(activities))
			]
			next_idx = argmax(
				[
					-min_dists[i] if i in remaining else float("-inf")
					for i in range(len(activities))
				]
			).astype(int)
			ordered.append(next_idx)
			remaining.remove(next_idx)

		ordered_activities = [activities[i] for i in ordered]
		print("\n📊 Incremental learning order:")
		for i, act in enumerate(ordered_activities, 1):
			print(f"  {act}: {activity_counts[act]:,} samples")

		return ordered_activities

	def get_fold(
		self,
		fold_idx: int,
		val_split: float = 0.2,
		normalize: bool = True,
		scaler: RobustScaler | None = None,
	) -> tuple[
		DataFrame,
		DataFrame | None,
		DataFrame | None,
		DataFrame,
		DataFrame | None,
		DataFrame,
		RobustScaler | None,
	]:
		"""Get train/val/test split for a specific fold."""
		if fold_idx >= self.n_folds:
			raise ValueError(f"fold_idx {fold_idx} >= n_folds {self.n_folds}")

		n_train_activities = self.initial_activities + fold_idx * self.test_size
		train_activities = self.activity_order[:n_train_activities]
		test_activities = self.activity_order[
			n_train_activities : n_train_activities + self.test_size
		]

		print(f"\n🔄 Fold {fold_idx}:")
		print(f"  Training activities (normal): {train_activities}")
		print(f"  Test activities (novel): {test_activities}")

		train_mask = self.df["activity"].isin(train_activities)
		test_mask = self.df["activity"].isin(test_activities)

		train_val_df = self.df[train_mask].copy()
		test_df = self.df[test_mask].copy()

		split_idx = int(len(train_val_df) * (1 - val_split))
		train_df = train_val_df.iloc[:split_idx].copy()
		val_df = train_val_df.iloc[split_idx:].copy() if val_split > 0 else None

		feature_cols = [col for col in self.data.columns if col != "id"]

		X_train = train_df[feature_cols].reset_index(drop=True)
		X_val = (
			val_df[feature_cols].reset_index(drop=True) if val_df is not None else None
		)
		X_test = test_df[feature_cols].reset_index(drop=True)

		y_train = train_df[["id", "activity"]].reset_index(drop=True)
		y_val = (
			val_df[["id", "activity"]].reset_index(drop=True)
			if val_df is not None
			else None
		)
		y_test = test_df[["id", "activity"]].reset_index(drop=True)

		print(f"  Train samples: {len(X_train):,}")
		if X_val is not None:
			print(f"  Val samples: {len(X_val):,}")
		print(f"  Test samples: {len(X_test):,}")

		if normalize:
			X_train, X_val, X_test, scaler = self.normalize_features(
				X_train, X_val, X_test, scaler=scaler
			)
		return X_train, X_val, X_test, y_train, y_val, y_test, scaler

## **TRIALS AND EVALUATION PHASE**:

In [139]:
def get_param_space(
	search_method: Literal["Random", "Annealing", "Genetic", "Bayesian"],
	use_log_dist: bool = True,
) -> dict:
	"""
	Get parameter space for ensemble hyperparameter optimization.

	Args:
		search_method: The optimization method to use
		use_log_dist: Whether to use log-uniform distributions for continuous params.
		used only for RandomSearchCV params.

	Returns:
		Parameter space dictionary appropriate for the search method
	"""
	if search_method == "Random":
		# Log-uniform distributions for better coverage
		if use_log_dist:
			return {  # Discrete uniform distributions
				"sequence_length": [50, 75, 100, 125, 150, 175, 200, 225],
				"hidden_size": [32, 48, 64, 80, 128, 256, 512],
				"n_layers": [1, 2, 3, 4],
				"dropout": [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5],
				"epochs": [20, 25, 30, 45, 50],
				"batch_size": [64, 128, 256],
				"learning_rate": [1e-4, 5e-4, 1e-3, 5e-3, 1e-2],
				"n_estimators": [100, 200, 300],
				"max_samples": [128, 256, 512],
				"contamination": [0.01, 0.05, 0.1, 0.15],
				"nu": [0.01, 0.05, 0.1, 0.2, 0.3],
				"gamma": [1e-4, 1e-3, 1e-2, 0.1, 0.5, 1.0, "scale"],
				"weights": [
					(0.5, 0.25, 0.25),
					(0.4, 0.3, 0.3),
					(0.33, 0.33, 0.34),
					(0.6, 0.2, 0.2),
				],
			}
		else:
			return {  # Discrete uniform distributions
				# LSTM-AE params
				"sequence_length": [50, 75, 100, 150, 200],
				"hidden_size": [32, 64, 128, 256],
				"n_layers": [1, 2, 3],
				"dropout": loguniform(0.1, 0.5),
				"epochs": [20, 30, 50],
				"batch_size": [64, 128, 256],
				"learning_rate": loguniform(1e-4, 1e-2),
				# Isolation Forest params
				"n_estimators": [100, 200, 300],
				"max_samples": [128, 256, 512],
				"contamination": loguniform(0.01, 0.15),
				# OC-SVM params
				"nu": loguniform(0.01, 0.3),
				"gamma": loguniform(1e-4, 1.0),
				# Ensemble weights
				"weights": [
					(0.5, 0.25, 0.25),
					(0.4, 0.3, 0.3),
					(0.33, 0.33, 0.34),
					(0.6, 0.2, 0.2),
				],
			}
	if search_method in ["Annealing", "Genetic"]:
		return {  # Using log-uniform for these methods as well
			"sequence_length": [50, 75, 100, 150, 200],
			"hidden_size": [32, 64, 128, 256],
			"n_layers": [1, 2, 3],
			"dropout": loguniform(0.1, 0.5),
			"epochs": [20, 30, 50],
			"batch_size": [64, 128, 256],
			"learning_rate": loguniform(1e-4, 1e-2),
			"n_estimators": [100, 200, 300],
			"max_samples": [128, 256, 512],
			"contamination": loguniform(0.01, 0.15),
			"nu": loguniform(0.01, 0.3),
			"gamma": loguniform(1e-4, 1.0),
			"weights": [
				(0.5, 0.25, 0.25),
				(0.4, 0.3, 0.3),
				(0.33, 0.33, 0.34),
				(0.6, 0.2, 0.2),
			],
		}
	elif search_method == "Bayesian":
		return {  # Bounds for Bayesian optimization (continuous parameters only)
			"sequence_length": (50, 200),
			"hidden_size": (32, 256),
			"n_layers": (1, 3),
			"dropout": (0.1, 0.5),
			"epochs": (20, 50),
			"batch_size": (64, 256),
			"learning_rate": (1e-4, 1e-2),
			"n_estimators": (100, 300),
			"max_samples": (128, 512),
			"contamination": (0.01, 0.15),
			"nu": (0.01, 0.3),
			"gamma_log": (-4, 0),  # log10(gamma), so 1e-4 to 1
			"weight_lstm": (0.2, 0.6),  # i'll derive other weights from this
		}
	raise ValueError(f"Unknown search method: {search_method}")


def train_search_method(
	training_data: DataFrame,
	train_targets: Series,
	search_type: Literal["Random", "Annealing", "Genetic", "Bayesian"],
	params_type: Literal["Continuous", "Discrete"] = "Continuous",
	n_iter: int = 100,
	cv: int = 4,
	verbose: int = 1,
	random_state: int = 42,
) -> RandomizedSearchCV | SimulatedAnnealingSearch | GeneticSearch | BayesianSearch:
	"""
	Factory function to create and train hyperparameter search method.

	Args:
		training_data: Training feature data
		train_targets: Training labels
		search_type: Type of search method to use
		n_iter: Number of iterations for the search
		cv: Number of cross-validation folds
		verbose: Verbosity level
		random_state: Random seed

	Returns:
		Fitted search object
	"""
	# Determine if using log distributions for Random search
	use_log_dist = search_type == "Random" and params_type == "Discrete"
	params = get_param_space(
		"Random" if use_log_dist else search_type, use_log_dist=use_log_dist
	)
	params["n_features"] = len(IMU_COLUMNS)
	if search_type == "Annealing":
		return SimulatedAnnealingSearch(
			param_space=params,
			n_iter=n_iter,
			initial_temp=1.0,
			cooling_rate=0.95,
			random_state=random_state,
		).fit(training_data, train_targets)

	if search_type == "Genetic":
		population_size = min(20, n_iter // 5)
		return GeneticSearch(
			param_space=params,
			population_size=population_size,
			n_generations=n_iter // population_size,
			mutation_rate=0.1,
			crossover_rate=0.8,
			random_state=random_state,
		).fit(training_data, train_targets)

	if search_type == "Bayesian":
		return BayesianSearch(
			param_bounds=params,
			n_iter=n_iter,
			random_state=random_state,
		).fit(training_data, train_targets)

	if search_type == "Random":
		return RandomizedSearchCV(
			estimator=NoveltyDetectionEnsemble(n_features=len(IMU_COLUMNS)),
			param_distributions=params,
			n_iter=n_iter,
			cv=cv,
			scoring=lambda est, X, y: score_function(est, X, y, X)["f1_score"],
			verbose=verbose,
			random_state=random_state,
			error_score="raise",
		).fit(training_data, train_targets)

	raise ValueError(f"Unknown search type: {search_type}")

In [140]:
def eval_search_method(
	dataset: IncrementalDataset,
	search_type: Literal["Random", "Annealing", "Genetic", "Bayesian"],
	log_distributions: bool = False,
	n_iter: int = 50,
	log_file: Path | str | None = None,
) -> list[dict]:
	"""
	Evaluate a hyperparameter search method across all dataset folds.

	Args:
		dataset: IncrementalDataset instance with prepared folds
		search_type: Type of search method to use
		n_iter: Number of iterations for the search
		log_file: Optional path to log file. If None, uses default naming

	Returns:
		List of evaluation metrics for each fold
	"""
	# Setup logging
	if log_file is None:
		log_file = Path(f"../reports/logs_{search_type.lower()}.log")

	log_file.parent.mkdir(parents=True, exist_ok=True)
	file_logger = configure_file_logger(log_file)

	all_fold_results = []

	for fold_idx in range(dataset.n_folds):
		X_train, X_val, X_test, y_train, y_val, y_test, scaler = dataset.get_fold(
			fold_idx, val_split=0.2, normalize=True
		)
		search = train_search_method(
			training_data=X_train,
			train_targets=Series([1] * len(X_train)),  # All training data is "normal"
			params_type="Discrete" if log_distributions else "Continuous",
			search_type=search_type,
			n_iter=n_iter,
			random_state=42,
		)
		file_logger.info(
			json.dumps(
				{
					k: (v if not isinstance(v, (np.integer, np.floating)) else float(v))
					for k, v in search.best_params_.items()
				},
				indent=2,
			)
		)
		all_fold_results.append(
			{  # Evaluate on test set (all test samples are "novel")
				"fold": fold_idx,
				"search_type": search_type,
				"best_params": search.best_params_,
				**score_function(
					NoveltyDetectionEnsemble(  # Final model with best params
						n_features=len(IMU_COLUMNS),
						**{
							k: v
							for k, v in search.best_params_.items()
							if k != "n_features"
						},
					).fit(X_train),
					X_train,
					Series([True] * len(X_test)),
					X_test,
				),
			}
		)
	return all_fold_results

In [141]:
# dataset = IncrementalDataset(
# 	Path("../data/PAMAP2_Dataset/Protocol/"),
# 	initial_activities=3,
# 	test_size=2,
# 	ordering_method="pca",
# )
# print("\n1. Running Random Search (discrete)...")
# random_results = eval_search_method(
# 	dataset=dataset,
# 	search_type="Random",
# 	n_iter=50,
# 	log_file=Path("../reports/logs_random_discrete.log"),
# )
# print("\n2. Running Random Search (log distributions)...")
# random_log_results = eval_search_method(
# 	dataset=dataset,
# 	search_type="Random",
# log_distributions=True,
# 	n_iter=50,
# 	log_file=Path("../reports/logs_random_log.log"),
# )
print("\n3. Running Simulated Annealing...")
annealing_results = eval_search_method(
	dataset=dataset,
	search_type="Annealing",
	n_iter=100,
	log_file=Path("../reports/logs_annealing.log"),
)
print("\n4. Running Genetic Algorithm...")
genetic_results = eval_search_method(
	dataset=dataset,
	search_type="Genetic",
	n_iter=100,
	log_file=Path("../reports/logs_genetic.log"),
)
print("\n5. Running Bayesian Optimization...")
bayesian_results = eval_search_method(
	dataset=dataset,
	search_type="Bayesian",
	n_iter=50,
	log_file=Path("../reports/logs_bayesian.log"),
)


3. Running Simulated Annealing...

🔄 Fold 0:
  Training activities (normal): [16, 13, 3]
  Test activities (novel): [5, 17]
  Train samples: 376,113
  Val samples: 94,029
  Test samples: 336,855
{'sequence_length': 50, 'hidden_size': 32, 'n_layers': 3, 'dropout': np.float64(0.18272261776066237), 'epochs': 30, 'batch_size': 64, 'learning_rate': np.float64(0.0005611516415334506), 'n_estimators': 100, 'max_samples': 128, 'contamination': np.float64(0.02757359293934948), 'nu': np.float64(0.03574712922600244), 'gamma': np.float64(0.003148911647956862), 'weights': (0.5, 0.25, 0.25), 'n_features': 30}


RuntimeError: Cannot clone object NoveltyDetectionEnsemble(contamination=np.float64(0.02757359293934948),
                         dropout=np.float64(0.18272261776066237), epochs=30,
                         gamma=np.float64(0.003148911647956862), hidden_size=32,
                         learning_rate=np.float64(0.0005611516415334506),
                         max_samples=128, n_estimators=100, n_features=30,
                         n_layers=3, nu=np.float64(0.03574712922600244),
                         sequence_length=50,
                         weights=(np.float64(0.5), np.float64(0.25),
                                  np.float64(0.25))), as the constructor either does not set or modifies parameter weights